In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
import torch.utils.data as data
from torchvision.datasets import ImageFolder

# Define constants
NUM_CLASSES = 4  # Number of classes (correct, partially correct, incorrect, none)
IMG_WIDTH, IMG_HEIGHT = 224, 224  # Input image dimensions for ResNeXt
BATCH_SIZE = 32
EPOCHS = 15
LEARNING_RATE = 0.001

# Load the pre-trained ResNeXt-50 model
resnext50_32x4d = models.resnext50_32x4d(pretrained=True)

# Freeze the base model's layers
for param in resnext50_32x4d.parameters():
    param.requires_grad = False

# Modify the classifier for your specific number of classes
num_ftrs = resnext50_32x4d.fc.in_features
resnext50_32x4d.fc = nn.Linear(num_ftrs, NUM_CLASSES)

# Define the transformation for the images
transform = transforms.Compose([
    transforms.Resize((IMG_WIDTH, IMG_HEIGHT)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the training and validation data
TRAINING_DIR = "C:/Users/Lenovo/Desktop/koi bhi/train/"
train_dataset = ImageFolder(TRAINING_DIR, transform=transform)
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

VALIDATION_DIR = "C:/Users/Lenovo/Desktop/koi bhi/test/"
val_dataset = ImageFolder(VALIDATION_DIR, transform=transform)
val_loader = data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Initialize the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnext50_32x4d.fc.parameters(), lr=LEARNING_RATE)

# Move the model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnext50_32x4d.to(device)

# Training loop
for epoch in range(EPOCHS):
    resnext50_32x4d.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = resnext50_32x4d(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    # Calculate average loss for the epoch
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{EPOCHS}] - Loss: {avg_loss:.4f}")

# Validation loop
resnext50_32x4d.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = resnext50_32x4d(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate validation accuracy
accuracy = 100 * correct / total
print(f"Validation Accuracy: {accuracy:.2f}%")

# Save the model for later use
torch.save(resnext50_32x4d.state_dict(), 'pose_classification_model_resnext50_32x4d.pth')


c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/15] - Loss: 0.3162
Epoch [2/15] - Loss: 0.0638
Epoch [3/15] - Loss: 0.0416
Epoch [4/15] - Loss: 0.0262
Epoch [5/15] - Loss: 0.0175
Epoch [6/15] - Loss: 0.0139
Epoch [7/15] - Loss: 0.0113
Epoch [8/15] - Loss: 0.0087
Epoch [9/15] - Loss: 0.0079
Epoch [10/15] - Loss: 0.0088
Epoch [11/15] - Loss: 0.0127
Epoch [12/15] - Loss: 0.0044
Epoch [13/15] - Loss: 0.0046
Epoch [14/15] - Loss: 0.0048
Epoch [15/15] - Loss: 0.0047
Validation Accuracy: 100.00%


In [2]:
!pip install torchvision

                                              0.0/1.2 MB ? eta -:--:--
     -                                        0.0/1.2 MB 1.3 MB/s eta 0:00:01
     --                                       0.1/1.2 MB 812.7 kB/s eta 0:00:02
     ----                                     0.1/1.2 MB 1.1 MB/s eta 0:00:01
     -----                                    0.2/1.2 MB 952.6 kB/s eta 0:00:02
     --------                                 0.3/1.2 MB 1.1 MB/s eta 0:00:01
     ----------                               0.3/1.2 MB 1.1 MB/s eta 0:00:01
     -------------                            0.4/1.2 MB 1.3 MB/s eta 0:00:01
     ---------------                          0.5/1.2 MB 1.3 MB/s eta 0:00:01
     ------------------                       0.5/1.2 MB 1.3 MB/s eta 0:00:01
     -------------------                      0.6/1.2 MB 1.3 MB/s eta 0:00:01
     --------------------                     0.6/1.2 MB 1.2 MB/s eta 0:00:01
     ---------------------                    0.7/1.2 MB 1.2 MB/s e


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip
